В данном датасете построим некоторую базовую модель, относительно предсказаний которой можно будет отталкиваться в оценке точности.

Устроим блиц-проверку алгоритмов sklearn по мотивам статьи https://habr.com/ru/post/475552/

По условиям соревнования, критерием оценки будет MAPE. Мы тоже будет его использовать.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV

# Загрузка тренировочного датасета

In [3]:
X = pd.read_csv('EDAv2_Train.zip')
y = X['price']
X.drop(columns='price', inplace=True)
X.head(1).T

,0
body_type,седан
brand,AUDI
color,чёрный
fuel_type,бензин
model_year,1990
n_doors,4
production_year,1991
vehicle_transmission,механика
engine_power,174.0
description,"Машина в приличном состоянии ,не гнилая не р..."


In [4]:
# y = pd.read_csv('EDAv2_Train.zip', usecols=['price'])
y.head(1)

0    200000.0
Name: price, dtype: float64

In [5]:
X_test = pd.read_csv('EDAv2_Test.zip')
X_test.head(1).T

,0
body_type,лифтбек
brand,SKODA
color,синий
description,"Все автомобили, представленные в продаже, прох..."
engine_displacement,1.2
engine_power,105.0
fuel_type,бензин
mileage,74000
model_year,2013
n_doors,5


# Проверим готовность данных

In [6]:
# оставим в тренировочном датасете только те бренды, которые есть в тестовом
brand_list = list(X_test['brand'].unique())
brand_mask = X['brand'].apply(lambda x: True if x in brand_list else False)
X = X[brand_mask]
y = y[brand_mask]
# и проверим это
display('Train brands:', X['brand'].nunique())
display('Test brands:', X_test['brand'].nunique())

'Train brands:'

12

'Test brands:'

12

In [7]:
# оставим в тренировочном датасете только б/у машины
used_cars_mask = list(X['n_owners'] != '0')
X = X[used_cars_mask]
y = y[used_cars_mask]
display(X.shape, y.shape)

(37841, 17)

(37841,)

В обоих датасетах есть пропуски в объёме двигателя. 
При этом, во время EDA было видно, что у объема высокая корреляция с мощностью двигателя.
Поэтому, в данной ситуации, для baseline удалим этот признак.

Также удалим признак model_year - высокая корреляция с production_year, который д.б. более важным по смыслу.

Признак full_model_name тоже пока что удалим - слишком много вариаций.

Удалим description - ничего хорошего мы с ним здесь не сделаем.

In [8]:
# склеим датасеты признаков
X['is_train'] = 1
X_test['is_train'] = 0
X_ = X.append(X_test)
# обработаем
columns_to_drop = ['engine_displacement', 'model_year', 'full_model_name', 'description']
X_.drop(columns=columns_to_drop, inplace=True)
X_ = pd.get_dummies(X_)
# разделим
X = X_[X_['is_train'] == 1].drop(columns=['is_train'])
X_test = X_[X_['is_train'] == 0].drop(columns=['is_train'])

# Тюним гиперпараметры моделей

In [9]:
seed=73

## Настройка параметров BaggingRegressor

In [10]:
# сетка параметров #1
br_grid = {'n_estimators': [50, 100, 200],
             'max_samples': [0.5, 0.75, 1.0],
             }
br_model = BaggingRegressor(n_jobs=-1, random_state=seed)
cv_folder = KFold(n_splits=3, shuffle=True, random_state=seed)
grid = GridSearchCV(br_model,
                    br_grid,
                    cv=cv_folder,
                    scoring='neg_mean_absolute_percentage_error',
                    refit=False,
                    verbose=2,
                    )
# grid.fit(X, y)
# display(grid.best_score_)
# display(grid.best_params_)

In [11]:
# сетка параметров #2
br_grid = {'n_estimators': [200, 300, 400],
             'max_samples': [0.8, 0.9, 1.0],
             }
br_model = BaggingRegressor(n_jobs=-1, random_state=seed)
cv_folder = KFold(n_splits=3, shuffle=True, random_state=seed)
grid = GridSearchCV(br_model,
                    br_grid,
                    cv=cv_folder,
                    scoring='neg_mean_absolute_percentage_error',
                    refit=False,
                    verbose=2,
                    )
# grid.fit(X, y)
# display(grid.best_score_)
# display(grid.best_params_)

In [12]:
# тюненая модель BR
br_model = BaggingRegressor(n_jobs=-1,
                            n_estimators=300,
                            max_samples=0.9)

## Настройка параметров ExtraTreesRegressor

In [13]:
# сетка параметров #1
et_grid = {'n_estimators': [50, 100, 200],
           'max_depth': [None, 10, 20],
           'max_features': [0.25, 0.3, 0.35, 0.4]
          }
et_model = ExtraTreesRegressor(n_jobs=-1, random_state=seed)
cv_folder = KFold(n_splits=3, shuffle=True, random_state=seed)
grid = GridSearchCV(et_model,
                    et_grid,
                    cv=cv_folder,
                    scoring='neg_mean_absolute_percentage_error',
                    refit=False,
                    verbose=2,
                    )
# grid.fit(X, y)
# display(grid.best_score_)
# display(grid.best_params_)

In [14]:
# сетка параметров #2
et_grid = {'n_estimators': [200, 300, 400, 500],
           'max_depth': [None],
           'max_features': [0.4, 0.5, 0.6, 0.7]
          }
et_model = ExtraTreesRegressor(n_jobs=-1, random_state=seed)
cv_folder = KFold(n_splits=3, shuffle=True, random_state=seed)
grid = GridSearchCV(et_model,
                    et_grid,
                    cv=cv_folder,
                    scoring='neg_mean_absolute_percentage_error',
                    refit=False,
                    verbose=2,
                    )
# grid.fit(X, y)
# display(grid.best_score_)
# display(grid.best_params_)

In [15]:
# сетка параметров #3
et_grid = {'n_estimators': [200, 300, 400],
           'max_depth': [None],
           'max_features': [0.7, 0.8, 0.9, 1.0]
          }
et_model = ExtraTreesRegressor(n_jobs=-1, random_state=seed)
cv_folder = KFold(n_splits=3, shuffle=True, random_state=seed)
grid = GridSearchCV(et_model,
                    et_grid,
                    cv=cv_folder,
                    scoring='neg_mean_absolute_percentage_error',
                    refit=False,
                    verbose=2,
                    )
# grid.fit(X, y)
# display(grid.best_score_)
# display(grid.best_params_)

In [16]:
# тюненая модель ET
et_model = ExtraTreesRegressor(n_jobs=-1,
                               n_estimators=300,
                               max_features=0.9)

## Настройка параметров RandomForestRegressor

In [17]:
# сетка параметров #1
rf_grid = {'n_estimators': [50, 100, 200],
           'max_depth': [None, 5, 10, 15],
           'max_features': [0.3, 0.35, 0.4]
          }
rf_model = RandomForestRegressor(n_jobs=-1, random_state=seed)
cv_folder = KFold(n_splits=3, shuffle=True, random_state=seed)
grid = GridSearchCV(rf_model,
                    rf_grid,
                    cv=cv_folder,
                    scoring='neg_mean_absolute_percentage_error',
                    refit=False,
                    verbose=2,
                    )
# grid.fit(X, y)
# display(grid.best_score_)
# display(grid.best_params_)

In [18]:
# сетка параметров #2
rf_grid = {'n_estimators': [200, 300, 400, 500],
           'max_depth': [None],
           'max_features': [0.4, 0.45, 0.5]
          }
rf_model = RandomForestRegressor(n_jobs=-1, random_state=seed)
cv_folder = KFold(n_splits=3, shuffle=True, random_state=seed)
grid = GridSearchCV(rf_model,
                    rf_grid,
                    cv=cv_folder,
                    scoring='neg_mean_absolute_percentage_error',
                    refit=False,
                    verbose=2,
                    )
# grid.fit(X, y)
# display(grid.best_score_)
# display(grid.best_params_)

In [19]:
# тюненая модель RF
rf_model = RandomForestRegressor(n_jobs=-1,
                                 n_estimators=400,
                                 max_features=0.5)

## Настройка параметров KNeighborsRegressor

In [20]:
# сетка параметров #1
kn_grid = {'n_neighbors': [3, 5, 7, 9],
          }
kn_model = KNeighborsRegressor(n_jobs=-1)
cv_folder = KFold(n_splits=3, shuffle=True, random_state=seed)
grid = GridSearchCV(kn_model,
                    kn_grid,
                    cv=cv_folder,
                    scoring='neg_mean_absolute_percentage_error',
                    refit=False,
                    verbose=2,
                    )
# grid.fit(X, y)
# display(grid.best_score_)
# display(grid.best_params_)

In [21]:
# сетка параметров #2
kn_grid = {'n_neighbors': [1, 2, 3, 4],
          }
kn_model = KNeighborsRegressor(n_jobs=-1)
cv_folder = KFold(n_splits=3, shuffle=True, random_state=seed)
grid = GridSearchCV(kn_model,
                    kn_grid,
                    cv=cv_folder,
                    scoring='neg_mean_absolute_percentage_error',
                    refit=False,
                    verbose=2,
                    )
# grid.fit(X, y)
# display(grid.best_score_)
# display(grid.best_params_)

In [22]:
# тюненая модель KNN
kn_model = KNeighborsRegressor(n_jobs=-1,
                               n_neighbors=3) # оставим 3 для стабильности

## Проверим тюненые модели (не совсем корректно, но хоть так)

In [23]:
# параметры разделения
test_size = 0.2

In [24]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size)
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

(30272, 56) (30272,)
(7569, 56) (7569,)


In [25]:
# список моделей-кандидатов
# закомментированы совсем плохие по результатам или расходящиеся
models = []
models.append(('KNR', kn_model))
models.append(('BAR', br_model))
models.append(('ETR', et_model))
models.append(('RFR', rf_model))

In [26]:
# таблица результатов
models_effcy = pd.DataFrame(index=[name for name, model in models])

split_score - чем меньше, тем лучше

cv_mean - чем меньше по модулю, тем лучше

In [27]:
# Оценивание эффективности каждой модели
for name, model in models:
    # оценка с отложенной частью
    m_fit = model.fit(X_train, np.ravel(y_train))
    y_predict = model.predict(X_valid)
    m_score = mean_absolute_percentage_error(y_valid, y_predict)
    models_effcy.loc[name, 'split_score'] = m_score
    # оценка кросс-валидацией по всем данным
    kfold = KFold(n_splits=5, shuffle=True)
    cv_results = cross_val_score(model, X, np.ravel(y.values), cv=kfold, scoring='neg_mean_absolute_percentage_error')
    models_effcy.loc[name, 'cv_mean'] = cv_results.mean()
    models_effcy.loc[name, 'cv_std'] = cv_results.std()
    print(name, model, 'complete')
display(models_effcy)

KNR KNeighborsRegressor(n_jobs=-1, n_neighbors=3) complete
BAR BaggingRegressor(max_samples=0.9, n_estimators=300, n_jobs=-1) complete
ETR ExtraTreesRegressor(max_features=0.9, n_estimators=300, n_jobs=-1) complete
RFR RandomForestRegressor(max_features=0.5, n_estimators=400, n_jobs=-1) complete


,split_score,cv_mean,cv_std
KNR,0.502311,-0.508792,0.006973
BAR,0.181080,-0.181931,0.004209
ETR,0.203105,-0.187899,0.002659
RFR,0.183567,-0.179099,0.004145


# Результаты для Kaggle

In [28]:
submission = pd.read_csv('sample_submission_empty.csv')

## Bagging

In [29]:
# обучим
br_model.fit(X, y)
# предскажем
y_test_predict = br_model.predict(X_test)
# запишем
submission['price'] = np.around(y_test_predict, decimals=-3)
submission.to_csv('sample_submission_br_tune_round.csv', index=False)

## Extra Trees

In [30]:
# обучим
et_model.fit(X, y)
# предскажем
y_test_predict = et_model.predict(X_test)
# запишем
submission['price'] = np.around(y_test_predict, decimals=-3)
submission.to_csv('sample_submission_et_tune_round.csv', index=False)

## Random Forest

In [31]:
# обучим
rf_model.fit(X, y)
# предскажем
y_test_predict = rf_model.predict(X_test)
# запишем
submission['price'] = np.around(y_test_predict, decimals=-3)
submission.to_csv('sample_submission_rf_tune_round.csv', index=False)

## kNN

In [32]:
# обучим
kn_model.fit(X, y)
# предскажем
y_test_predict = kn_model.predict(X_test)
# запишем
submission['price'] = np.around(y_test_predict, decimals=-3)
submission.to_csv('sample_submission_kn_tune_round.csv', index=False)